## Imports

In [1]:
import numpy as np
import pprint
p = pprint.PrettyPrinter(indent=4).pprint

### Generate Random Inputs

Before we can create the optimization workflow, we first generate some random inputs. Then, we define the hierarchy level and select a subset of the product combinations to perform optimization on.

In [2]:
print('hello world')

hello world


In [3]:
hierarchy = {
    1: ('bulk', 'bag'),
    2: ('mesh', 'pellet', 'crumble'),
    3: ('cow', 'chicken', 'pig')
}

from itertools import product

all_possible_combinations = [
    '-'.join(prod) for prod in product(*hierarchy.values())
]

from random import sample, seed, randint

seed(10)
product_list = sorted(sample(all_possible_combinations, 3))

print(product_list)

['bag-crumble-cow', 'bag-pellet-chicken', 'bulk-mesh-chicken']


Per the model, each product combination will have its own list of customers and factories. Here we will just define the size of each combinations. We will also sample some factory names

In [4]:
seed(10)

factory_list = ['fa', 'fb', 'fc']
customer_list = ['ca', 'cb', 'cc', 'cd']

factory_sizes, customer_sizes, factory_names_, customer_names_ = {}, {}, {}, {}

for product in product_list: # Generate random size of factory and customer per product
    factory_sizes[product] = randint(1, 2)
    customer_sizes[product] = randint(2, 3)
    factory_names_[product] = sorted(sample(factory_list, factory_sizes[product]))
    customer_names_[product] = sorted(sample(customer_list, customer_sizes[product]))
    
print(f'{factory_sizes = }')
print(f'{customer_sizes = }')
print(f'{factory_names_ = }')
print(f'{customer_names_ = }')

factory_sizes = {'bag-crumble-cow': 1, 'bag-pellet-chicken': 2, 'bulk-mesh-chicken': 2}
customer_sizes = {'bag-crumble-cow': 3, 'bag-pellet-chicken': 3, 'bulk-mesh-chicken': 2}
factory_names_ = {'bag-crumble-cow': ['fb'], 'bag-pellet-chicken': ['fa', 'fc'], 'bulk-mesh-chicken': ['fa', 'fb']}
customer_names_ = {'bag-crumble-cow': ['ca', 'cb', 'cd'], 'bag-pellet-chicken': ['ca', 'cb', 'cc'], 'bulk-mesh-chicken': ['ca', 'cb']}


From the above data, we now randomly generate the logistics cost. The customers will represent the rows while the factories represent the columns 

In [5]:
seed(10)
inbound_cost_per_product = {}
outbound_cost_per_product = {}

for product in product_list:
    inbound_cost_per_product[product] = np.random.randint(
        10, 100, factory_sizes[product])
    outbound_cost_per_product[product] = np.random.randint(
        10, 100, (customer_sizes[product], factory_sizes[product]))
   
print('\033[1minbound_cost_per_product = \033[0;0m'); p(inbound_cost_per_product)
print('\033[1moutbound_cost_per_product = \033[0;0m'); p(outbound_cost_per_product)

inbound_cost_per_product = 
{   'bag-crumble-cow': array([83]),
    'bag-pellet-chicken': array([91, 21]),
    'bulk-mesh-chicken': array([70, 37])}
outbound_cost_per_product = 
{   'bag-crumble-cow': array([[57],
       [72],
       [89]]),
    'bag-pellet-chicken': array([[71, 85],
       [48, 57],
       [35, 66]]),
    'bulk-mesh-chicken': array([[27, 55],
       [93, 77]])}


For completeness, we will also generate some numbers for efficiency.

In [6]:
seed(10)
efficiency = {}

for product in product_list:
    efficiency[product] = np.random.randint(70, 100, factory_sizes[product])/100
    

print('\033[1mefficiency = \033[0;0m'); p(efficiency)

efficiency = 
{   'bag-crumble-cow': array([0.8]),
    'bag-pellet-chicken': array([0.98, 0.77]),
    'bulk-mesh-chicken': array([0.79, 0.72])}


# The Model

Now that we have all the necessary data, we can start assembling the linear program

## Objective Vector

To build the objective vector, we need to concatenate both the inbound and outbound cost vector. 

**Inbound cost vector takes from the inbound cost product, which is a dictionary with the name of product as keys and the array of inbound cost as values.**

**Outbound cost vector takes from the outbound cost per product, which is a dictionary with the name of the product as keys and the 2d array (customer x factory) of outbound cost as values**.

In [7]:
inbound_cost_per_product

{'bag-crumble-cow': array([83]),
 'bag-pellet-chicken': array([91, 21]),
 'bulk-mesh-chicken': array([70, 37])}

In [8]:
inbound_cost_vector = np.hstack(list(
    inbound_cost_per_product.values()))  # Unpack cost to inbound section

_lst_ = [mat.flatten('F') for mat in outbound_cost_per_product.values()
         ]  # Flatten the outbound_cost_per_product

outbound_cost_vector = np.hstack(_lst_)
#del _lst_  # Unpack cost to outbound section

objective_vector = np.hstack([inbound_cost_vector, outbound_cost_vector])

print(f'{inbound_cost_vector = }')
print(f'{outbound_cost_vector = }')
print(f'{objective_vector = }')

### Check dimension
sumF = sum(factory_sizes.values())
sumC = sum(customer_sizes.values())
sumFC = sum(
    np.array(list(factory_sizes.values())) *
    np.array(list(customer_sizes.values())))

assert len(
    inbound_cost_vector
) == sumF, "Inbound cost vector not the right shape (1, ∑|F|)"
assert len(
    outbound_cost_vector
) == sumFC, "Outbound cost vector not the right shape (1, ∑|FxC|)"

print(f'\n{sumF = }, {sumFC = }')
print(f'objective_cost_vector_len = {sumF+sumFC}')

inbound_cost_vector = array([83, 91, 21, 70, 37])
outbound_cost_vector = array([57, 72, 89, 71, 48, 35, 85, 57, 66, 27, 93, 55, 77])
objective_vector = array([83, 91, 21, 70, 37, 57, 72, 89, 71, 48, 35, 85, 57, 66, 27, 93, 55,
       77])

sumF = 5, sumFC = 13
objective_cost_vector_len = 18


In [9]:
inbound_cost_vector.shape == (5, )

True

## Constraints Matrix

Here, we will now build the Demand Matrix, Product Capacity Matrix, Product Supply Matrix, Joint Capacity Matrix, Joint Supply Matrix.

### Demand Matrix

In [10]:
from scipy.linalg import block_diag
from sympy import Matrix

In [12]:
demand_OB = block_diag(*[
    np.tile(np.eye(customer_sizes[product]), reps=factory_sizes[product])
    for product in product_list
]) # Demand Outbound Block

assert demand_OB.shape == (
    sumC,
    sumFC), 'Outbound block of demand matrix not the right shape (∑|C|, ∑|FxC|)'

demand_IB = np.zeros(
    (sumC,
     sumF)) # Demand Inbound Block

assert demand_IB.shape == (
    sumC,
    sumF), 'Inbound block of demand matrix not the right shape (∑|C|, ∑|F|)'

In [ ]:
np.split

In [31]:
demand_OB

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.]])

### Combination Matrix

We are going to create product block where each F_u will contain all the factories. Then suppose if a factory is not F_u then we leave the column to all zero and do not leave a space. So first, we have to write a function that insert [] if the factory does not exist

In [13]:
gen = (i for i in range(1, 5))

conds = [True, False, True, True, False]

[next(gen) if cond else 0 for cond in conds]

[1, 0, 2, 3, 0]

In [14]:
### Function to insert outbound blank, positive denotes the thing that will be insert if True
insert_outblank = lambda factory_list, factory_sublist, positive: [
    positive if factory in factory_sublist else [] for factory in factory_list
]

insert_inblank = lambda factory_list, factory_sublist, positive: [
    next(positive) if factory in factory_sublist else []
    for factory in factory_list
]

efficiency_val = (elem for elem in np.hstack(list(efficiency.values()))
                  )  # Add a generator to do the job

In [15]:
customer_sizes

{'bag-crumble-cow': 3, 'bag-pellet-chicken': 3, 'bulk-mesh-chicken': 2}

In [16]:
### List of blocks before doing block diagonal
_inblock_ = [
    insert_inblank(factory_list,
                 factory_sublist=factory_names_[product],
                 positive=efficiency_val)
    for product in product_list
]
_outblock_ = [
    insert_outblank(factory_list,
                 factory_sublist=factory_names_[product],
                 positive=[1] * customer_sizes[product])
    for product in product_list
]

print(f'{_inblock_ = }\n')
print(f'{_outblock_ = }')

_inblock_ = [[[], 0.8, []], [0.98, [], 0.77], [0.79, 0.72, []]]

_outblock_ = [[[], [1, 1, 1], []], [[1, 1, 1], [], [1, 1, 1]], [[1, 1], [1, 1], []]]


In [17]:
pcap_IB = block_diag(*[block_diag(*block)
                       for block in _inblock_])  #Product Capacity Inbound Block

assert pcap_IB.shape == (
    len(factory_list) * len(product_list), sumF
), 'Inbound block of product capacity matrix not the right shape'

pcap_OB = block_diag(*[block_diag(*block)
                       for block in _outblock_])  #Product Capacity Outbound Block

assert pcap_OB.shape == (
    len(factory_list) * len(product_list), sumFC
), 'Outbound block of product capacity matrix not the right shape'

### Extract the list of block matrix in product_inblock
product_inblock = dict(zip(product_list, np.split(pcap_IB, len(product_list), axis = 0)))
print('\033[1mproduct_inblock = \033[0m')
p(product_inblock)

### Extract the list of block matrix in product_outblock
product_outblock = dict(zip(product_list, np.split(pcap_OB, len(product_list), axis = 0)))
print('\n\033[1mproduct_outblock = \033[0m')
p(product_outblock)

product_inblock = 
{   'bag-crumble-cow': array([[0. , 0. , 0. , 0. , 0. ],
       [0.8, 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. ]]),
    'bag-pellet-chicken': array([[0.  , 0.98, 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.77, 0.  , 0.  ]]),
    'bulk-mesh-chicken': array([[0.  , 0.  , 0.  , 0.79, 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.72],
       [0.  , 0.  , 0.  , 0.  , 0.  ]])}

product_outblock = 
{   'bag-crumble-cow': array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]),
    'bag-pellet-chicken': array([[0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0.]]),
    'bulk-mesh-chicken': array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.,

In [18]:
product_outblock['bag-pellet-chicken']

array([[0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0.]])

### Product Supply Matrix

This is define similarly to the product capacity matrix, so we're not going to construct this again to save space.

### Joint Capacity Matrix

To do this, we first generate the random combination for capacity

In [19]:
from random import choices

In [20]:
seed(5)
### Generate Random Joint Combination
joint_capacity_combination = [
    sorted(sample(product_list, k=randint(1, len(product_list)))) for i in range(3)
]
print(f'\033[1mjoint_capacity_combination\033[0m = {joint_capacity_combination}')

joint_capacity_combination = [['bag-crumble-cow', 'bag-pellet-chicken', 'bulk-mesh-chicken'], ['bag-crumble-cow', 'bulk-mesh-chicken'], ['bag-crumble-cow']]


We create a function to add all the combination together into a matrix

In [21]:
### Function to add the combination
joint_combination_to_matrix = lambda combination, product_block: np.sum(
    [product_block[product] for product in combination], axis=0)

# Stack each of the submatrix to create one big matrix
joint_capacity_outmatrix = np.vstack([
    joint_combination_to_matrix(combination, product_outblock)
    for combination in joint_capacity_combination
])

# Now we need to strip the all zeros rows from the matrix,
# although we might to do this after we append it with the inbound block
joint_capacity_outmatrix = joint_capacity_outmatrix[
    ~np.all(joint_capacity_outmatrix == 0, axis=1)]

assert joint_capacity_outmatrix.shape[
    1] == sumFC, 'Joint Capacity Matrix not the right shape'

joint_capacity_inmatrix = np.zeros((joint_capacity_outmatrix.shape[0], sumF))

In [22]:
joint_capacity_inmatrix

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

### Joint Supply Matrix

The joint supply matrix is defined in similar ways as the joint capacity matrix. The only different is that we now have to repeat the same process for the inbound block. First, we

In [23]:
seed(4)
### Generate Random Joint Combination
joint_supply_combination = [
    sorted(sample(product_list, k=randint(1, len(product_list)))) for i in range(3)
]
print(f'\033[1mjoint_supply_combination\033[0m = {joint_supply_combination}')

joint_supply_combination = [['bag-pellet-chicken'], ['bulk-mesh-chicken'], ['bag-crumble-cow', 'bag-pellet-chicken']]


In [24]:
# Stack each of the submatrix to create one big matrix
joint_supply_outmatrix = np.vstack([
    joint_combination_to_matrix(combination, product_outblock)
    for combination in joint_supply_combination
])

# Now we need to strip the all zeros rows from the matrix,
# although we might to do this after we append it with the inbound block
joint_supply_outmatrix = joint_supply_outmatrix[
    ~np.all(joint_supply_outmatrix == 0, axis=1)]

assert joint_supply_outmatrix.shape[
    1] == sumFC, 'Joint Supply Matrix columns dimension not the right shape'

In [25]:
# Stack each of the submatrix to create one big matrix
joint_supply_inmatrix = np.vstack([
    joint_combination_to_matrix(combination, product_inblock)
    for combination in joint_supply_combination
])

# Now we need to strip the all zeros rows from the matrix,
# although we might to do this after we append it with the inbound block
joint_supply_inmatrix = joint_supply_inmatrix[
    ~np.all(joint_supply_inmatrix == 0, axis=1)]

assert joint_supply_inmatrix.shape[
    1] == sumF, 'Joint Inbound Supply Matrix columns dimension incorrect'

assert joint_supply_inmatrix.shape[0] == joint_supply_outmatrix.shape[
    0], 'Joint Supply Matrix row dimension incorrect'

In [26]:
joint_supply_inmatrix

array([[0.  , 0.98, 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.77, 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.79, 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.72],
       [0.  , 0.98, 0.  , 0.  , 0.  ],
       [0.8 , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.77, 0.  , 0.  ]])

# ----------------------------- The End -------------------------------